In [13]:
from __future__ import print_function

import glob
import nibabel
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib
import skimage.transform
import numpy as np

from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider
import ipywidgets as widgets
matplotlib.rcParams['figure.figsize'] = (20, 10)

In [14]:
path = Path('.')

In [7]:
ge3t.shape, singapore.shape, ultrecht.shape

((256, 256, 176), (256, 256, 192), (256, 256, 192))

In [10]:
patients = glob.glob(str(path / '*' / '*'))
print(f"{len(patients)} patients found.")

60 patients found.


In [ ]:
def show_image(patient):
    patient = patients[patient]
    print(patient)
    t1 = nibabel.load(path / patient / 'pre' / 'T1.nii.gz').get_fdata()
    flair = nibabel.load(path / patient / 'pre' / 'FLAIR.nii.gz').get_fdata()
    gt = nibabel.load(path / patient / 'wmh.nii.gz').get_fdata()
    assert t1.shape == flair.shape and t1.shape == gt.shape
    def f(s):
        plt.subplot(1, 3, 1)
        plt.title('T1')
        plt.imshow(t1[:, :, s])
        plt.subplot(1, 3, 2)
        plt.title('FLAIR')
        plt.imshow(flair[:, :, s])
        plt.subplot(1, 3, 3)
        plt.title('Ground Truth')
        plt.imshow(gt[:, :, s])
        plt.show()
    return interact(f, s=IntSlider(min=0, max=t1.shape[-1]-1, value=0))
interact_manual(show_image, patient=IntSlider(min=0, max=len(patients)-1, value=0))

In [ ]:
images = np.load('images.npy')
gts = np.load('gts.npy')

In [ ]:
def show_image(patient):
    print(patients[patient])
    def f(s):
        plt.subplot(1, 3, 1)
        plt.title('T1')
        plt.imshow(images[patient, :, :, s, 0])
        plt.subplot(1, 3, 2)
        plt.title('FLAIR')
        plt.imshow(images[patient, :, :, s, 1])
        plt.subplot(1, 3, 3)
        plt.title('Ground Truth')
        plt.imshow(gts[patient, :, :, s])
        plt.show()
    return interact(f, s=IntSlider(min=0, max=images.shape[-2]-1, value=0))
interact_manual(show_image, patient=IntSlider(min=0, max=len(patients)-1, value=0))

In [4]:
import skimage
import skimage.io
import skimage.morphology

In [46]:
expected_shape = (200, 200, 48)
disk = skimage.morphology.disk(2.5)

def generate_samples(save_dir, data, n0, sample_size=16, gt=False, preprocess=False):
    _, _, vol, _ = data.shape
    if vol % sample_size != 0:
        print('Error, data and sample size not divisble')
        return
    saveN0 = n0
    n0 = str(n0)
    nb_samples = vol // sample_size
    imgs = []
    for sample_idx in range(nb_samples):
        curr_sample = data[:, :, sample_idx * sample_size: (sample_idx + 1) * sample_size, :]
        if gt:
            # This lead to /labels/50_0.npy
            np.save(save_dir + '/labels/' + n0 + '_' + str(sample_idx) + '.npy', curr_sample)
        else:
            if not preprocess:
                np.save(save_dir + '/data/' + n0 + '_' + str(sample_idx) + '.npy', curr_sample)
            else:
                np.save(save_dir + '/data/' + n0 + '_' + str(sample_idx) + '_preprocessed.npy', curr_sample)
        imgs.append(sample_idx)
    return np.array(imgs)

def generate_data(save_dir):
    dic = {}
    for patient in patients:
        print(patient)
        t1 = nibabel.load(path / patient / 'pre' / 'T1.nii.gz').get_fdata()
        fl = nibabel.load(path / patient / 'pre' / 'FLAIR.nii.gz').get_fdata()
        gt = nibabel.load(path / patient / 'wmh.nii.gz').get_fdata() != 0
        
        _, _, t1_c = t1.shape
        _, _, fl_c = fl.shape
        _, _, gt_c = gt.shape
        
        if t1_c == 83:
            t1 = t1[:, :, 2:82]
            t1_c = 80
        if fl_c == 83:
            fl = fl[:, :, 2:82]
            fl_c = 80
        if gt_c == 83:
            gt = gt[:, :, 2:82]
            gt_c = 80

        t1 = skimage.transform.resize(t1, (200, 200, t1_c))
        fl = skimage.transform.resize(fl, (200, 200, fl_c))
        gt = skimage.transform.resize(gt, (200, 200, gt_c))
        

        slices_morph = None
        for s in range(fl_c):
            flair_slice = fl[:, :, s]
            
        
            morph = skimage.morphology.dilation(flair_slice, disk)
            top_hat = skimage.util.invert(morph - flair_slice)
            top_hat = top_hat.reshape((200, 200, 1))
            if slices_morph is None:
                slices_morph = top_hat
            else:
                slices_morph = np.concatenate([slices_morph, top_hat], axis=-1)

        slices_morph = np.array(slices_morph)
        
        slices_morph = slices_morph.reshape((200, 200, fl_c, 1))

        t1 = t1.reshape((200, 200, t1_c, 1))
        fl = fl.reshape((200, 200, fl_c, 1))
        gt = gt.reshape((200, 200, gt_c, 1))
        
        img = np.concatenate([t1, fl], axis=-1)
        pre_processed = np.concatenate([img, slices_morph], axis=-1)
        
        n0 = patient.split('/')[1]
        
        dic[int(n0)] = (generate_samples(save_dir, img, n0, sample_size=16, gt=False, preprocess=False))
        generate_samples(save_dir, gt, n0, sample_size=16, gt=True, preprocess=False)
        generate_samples(save_dir, pre_processed, n0, sample_size=16, gt=False, preprocess=True)
    return dic

In [47]:
ids = generate_data('.')

GE3T/126
GE3T/144
GE3T/100
GE3T/106
GE3T/107
GE3T/108
GE3T/102
GE3T/137
GE3T/101
GE3T/114
GE3T/110
GE3T/116
GE3T/112
GE3T/132
GE3T/105
GE3T/109
GE3T/113
GE3T/104
GE3T/115
GE3T/103
Utrecht/27
Utrecht/37
Utrecht/21
Utrecht/49
Utrecht/6
Utrecht/4
Utrecht/17
Utrecht/35
Utrecht/29
Utrecht/8
Utrecht/23
Utrecht/33
Utrecht/2
Utrecht/31
Utrecht/39
Utrecht/25
Utrecht/19
Utrecht/11
Utrecht/0
Utrecht/41
Singapore/67
Singapore/55
Singapore/59
Singapore/53
Singapore/62
Singapore/63
Singapore/51
Singapore/50
Singapore/54
Singapore/69
Singapore/60
Singapore/56
Singapore/65
Singapore/66
Singapore/64
Singapore/58
Singapore/68
Singapore/57
Singapore/61
Singapore/52


In [49]:
x_train = np.load('../models/2d/X_train.npy')
x_test = np.load('../models/2d/X_test.npy')
y_train = np.load('../models/2d/Y_train.npy')
y_test = np.load('../models/2d/Y_test.npy')
transformed_x_train = [path[0].split('/')[4] for path in x_train]
transformed_y_train = [path.split('/')[4] for path in y_train]
transformed_x_test = [path[0].split('/')[4] for path in x_test]
transformed_y_test = [path.split('/')[4] for path in y_test]
# np.save('X_train.npy', transformed_x_train)
# np.save('X_test.npy', transformed_x_test)
# np.save('Y_train.npy', transformed_y_train)
# np.save('Y_test.npy', transformed_y_test)

In [69]:
all_slices = []
for k, v in ids.items():
    for idx in v:
        all_slices.append([int(k), idx])
all_slices = np.array(all_slices)

In [70]:
np.random.seed(42)

In [71]:
np.random.shuffle(all_slices)

In [72]:
from sklearn.model_selection import train_test_split

In [73]:
X_train_slices, X_test_slices = train_test_split(all_slices, test_size=0.20, random_state=42)

In [75]:
np.save('X_train_slices.npy', X_train_slices)
np.save('X_test_slices.npy', X_test_slices)